---
title: State data pipeline
---

Here we define state data to be the complementary data of the magnetic field data, useful for the analysis of the magnetic field data. It may include:

- Plasma data
- Satellite location data